In [1]:
import os
import glob
from tqdm import tqdm
from PIL import  Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.transforms import  ToTensor, Compose, Resize, Normalize


from dcnn import DCNN
from utils import encode_label_from_path, VehicleColorDataset, Logger

In [2]:
path = "/home/datdq/1WorkSpace/Dataset_General/COLOR_TYPE/train"
image_list = glob.glob(os.path.join(path, '**', '*.jpg'), recursive=True)
class_list = [encode_label_from_path(item) for item in image_list]
print("Total Images: ", len(image_list))
print("Total Classes: ", len(set(class_list)))
# Splitting the Dataset
x_train, x_test , y_train , y_test = train_test_split(image_list, class_list, train_size= 0.5 , stratify=class_list , shuffle=True, random_state=42)
transforms=Compose([Resize((227, 227)), 
                    ToTensor(),
                    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
# Create DataLoader for training and testing datasets
train_dataset = VehicleColorDataset( x_train , y_train , transforms)
train_data_loader = DataLoader(train_dataset,batch_size=115, shuffle=True, num_workers=4)
test_dataset = VehicleColorDataset(x_test, y_test,transforms)
test_data_loader = DataLoader(test_dataset, batch_size=115, num_workers=4)


Total Images:  18246
Total Classes:  8


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DCNN(num_classes=8).to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(), lr=1e-4)
epochs = 10
logger = Logger(log_dir='logs', name='DCNN_color', chkpt_interval=1)

In [ ]:
for epoch in range(epochs):
    with tqdm(train_data_loader, unit="batch") as tepoch:
        model.train()
        running_loss = 0
        batch_ = 0
        for X, y in tepoch:
            tepoch.set_description(f"Train | Epoch {epoch}")
            X, y = X.to('cuda'), y.to('cuda')

            pred = model(X)
            loss_value = loss_fn(pred, y)

            loss_value.backward()
            opt.step()
            opt.zero_grad()

            running_loss += loss_value.item()
            batch_ += 1
            tepoch.set_postfix(loss=running_loss / batch_)

        logger.log('train_loss', running_loss / batch_)

    # === Evaluation ===
    with torch.no_grad():
        model.eval()
        with tqdm(test_data_loader, unit="batch") as tepoch:
            tepoch.set_description(f"Test | Epoch {epoch}")
            correct = 0
            total = 0
            n_batch = 0
            running_loss = 0

            for X, y in tepoch:
                X, y = X.to('cuda'), y.to('cuda')
                pred = model(X)
                loss_value = loss_fn(pred, y)
                pred_class = pred.argmax(dim=1)

                running_loss += loss_value.item()
                correct += (pred_class == y).sum().item()
                total += y.size(0)
                n_batch += 1

                tepoch.set_postfix(
                    loss=running_loss / n_batch,
                    accuracy=(correct / total) * 100
                )

            logger.log('test_loss', running_loss / n_batch)
            logger.log('test_acc', correct / total)
            logger.checkpoint(model, correct / total)

Train | Epoch 0: 100%|██████████| 80/80 [00:20<00:00,  3.98batch/s, loss=0.271]
Test | Epoch 0: 100%|██████████| 80/80 [00:08<00:00,  9.54batch/s, accuracy=97.1, loss=0.0926]


New best accuracy: 0.9710 (prev: 0.0000), saving model...
Saving Model...


Train | Epoch 1: 100%|██████████| 80/80 [00:20<00:00,  3.98batch/s, loss=0.0625]
Test | Epoch 1: 100%|██████████| 80/80 [00:08<00:00,  9.50batch/s, accuracy=97.8, loss=0.0716]


New best accuracy: 0.9782 (prev: 0.9710), saving model...
Saving Model...


Train | Epoch 2: 100%|██████████| 80/80 [00:19<00:00,  4.02batch/s, loss=0.0389]
Test | Epoch 2: 100%|██████████| 80/80 [00:08<00:00,  9.43batch/s, accuracy=98.2, loss=0.062] 


...

New best accuracy: 0.9912 (prev: 0.9820), saving model...
Saving Model...


Train | Epoch 5: 100%|██████████| 80/80 [00:20<00:00,  3.98batch/s, loss=0.01]   
Test | Epoch 5: 100%|██████████| 80/80 [00:08<00:00,  9.46batch/s, accuracy=99.3, loss=0.0295]


New best accuracy: 0.9925 (prev: 0.9912), saving model...
Saving Model...


Train | Epoch 6: 100%|██████████| 80/80 [00:20<00:00,  3.93batch/s, loss=0.0117] 
Test | Epoch 6: 100%|██████████| 80/80 [00:08<00:00,  9.59batch/s, accuracy=99.2, loss=0.0352]
Train | Epoch 7: 100%|██████████| 80/80 [00:20<00:00,  3.93batch/s, loss=0.0137] 
Test | Epoch 7: 100%|██████████| 80/80 [00:08<00:00,  9.43batch/s, accuracy=99, loss=0.0345]  
Train | Epoch 8: 100%|██████████| 80/80 [00:20<00:00,  3.85batch/s, loss=0.0356]
Test | Epoch 8: 100%|██████████| 80/80 [00:08<00:00,  9.60batch/s, accuracy=97.9, loss=0.0701]
Train | Epoch 9: 100%|██████████| 80/80 [00:20<00:00,  3.93batch/s, loss=0.0228]
Test | Epoch 9: 100%|██████████| 80/80 [00:08<00:00,  9.22batch/s, accuracy=98.6, loss=0.0569]


: 